Import dependencies

In [1]:
import landmark_detector as ld
import os
import random
import numpy as np
import keras
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from keras import layers

Set variables

In [2]:
words = ['deaf', 'eat', 'fish', 'friend', 'like', 'milk', 'nice', 'no', 'orange', 'teacher', 'want', 'what', 'where', 'yes']
select_words = ['no', 'eat', 'teacher', 'want', 'fish']
path = '../preprocessing/dataset/'
num_features = 126
model_name = 'draft_model'
fps = 20

Get data from mediapipe


In [3]:
def get_data(words, path, detector_path):
    detector = ld.get_detector(detector_path)

    X = []
    y = []

    num_videos = 0
    highest_frame = 0

    bad_videos = 0

    for word in tqdm(words):
        word_path = os.path.join(path, word)
        
        video_files = [f for f in os.listdir(word_path) if f.endswith('.mp4')]
        
        for video_file in tqdm(video_files, desc=word):
            video_path = os.path.join(word_path, video_file)
            
            try:
                video_X = []
                landmarks, current_frames = ld.get_landmarks(video_path, detector)
                
                if len(landmarks) == 0:
                    bad_videos+=1
                    continue
                
                if current_frames > highest_frame:
                    highest_frame = current_frames
                
                for frame in range(len(landmarks)):
                    features = np.array(landmarks[frame]).flatten()
                    video_X.append(features)
                
                X.append(video_X)
                y.append(words.index(word))
                num_videos += 1

            except Exception as e:
                print(f"Error processing video {video_file}: {e}")
                continue 

    return X, y, num_videos, highest_frame, bad_videos

X, y, num_videos, highest_frame, bad_videos = get_data(select_words, path, '../models/hand_landmarker.task')

print('Number of videos:', num_videos)
print('Highest frame:', highest_frame)
print('Videos with no landmarkers detected: ', bad_videos)


  0%|          | 0/5 [00:00<?, ?it/s]

no:   0%|          | 0/27 [00:00<?, ?it/s]

eat:   0%|          | 0/28 [00:00<?, ?it/s]

teacher:   0%|          | 0/27 [00:00<?, ?it/s]

want:   0%|          | 0/27 [00:00<?, ?it/s]

fish:   0%|          | 0/29 [00:00<?, ?it/s]

Number of videos: 138
Highest frame: 47
Videos with no landmarkers detected:  0


Padding and Masking X

In [4]:
def padX(X, num_videos, highest_frame, num_features):
    padded_X = np.zeros((num_videos, highest_frame, num_features))
    mask = np.ones((num_videos, highest_frame, num_features)) 
    for i in range(num_videos):
        video = X[i]
        for j in range(len(video)):
            frame = video[j]
            if len(frame) < num_features:
                padded_X[i, j, :] = np.pad(frame, (0, num_features - len(frame)), 'constant')
                mask[i, j, len(frame):] = 0
            else:
                padded_X[i, j, :] = frame
        if len(video) < highest_frame:
            mask[i, len(video):, :] = 0

    return padded_X, mask

padded_X, mask = padX(X, num_videos, highest_frame, num_features)
print(padded_X.shape)

(138, 47, 126)


Create model

In [5]:
model = keras.Sequential()

model.add(keras.Input(shape=(highest_frame, num_features)))
model.add(layers.Masking(mask_value=0.0))
model.add(layers.LSTM(64))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(select_words), activation='sigmoid'))


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 47, 126)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,141 (199.77 KB)

 Trainable params: 51,141 (199.77 KB)

 Non-trainable params: 0 (0.00 B)

Train model

In [10]:
X_train, X_temp, y_train, y_temp = train_test_split(padded_X, y, test_size=0.3, random_state=42) 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) 


X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

model.save(f'../models/{model_name}.keras')

with open(f"../models/{model_name}.env", "w") as file:
    file.write(f"MAX_FRAMES={highest_frame}\n")
    file.write(f"NUM_FEATURES={num_features}\n")
    file.write(f"WORDS={",".join(select_words)}\n")
    file.write(f"FPS={fps}\n")

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - accuracy: 0.2240 - loss: 1.6118 - val_accuracy: 0.2381 - val_loss: 1.6264
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3164 - loss: 1.5839 - val_accuracy: 0.3333 - val_loss: 1.5982
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4349 - loss: 1.4899 - val_accuracy: 0.3333 - val_loss: 1.5911
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4180 - loss: 1.4893 - val_accuracy: 0.2857 - val_loss: 1.5788
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3503 - loss: 1.4706 - val_accuracy: 0.3810 - val_loss: 1.5592
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4115 - loss: 1.4346 - val_accuracy: 0.4286 - val_loss: 1.5525
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4479 - loss: 1.4124 - val_accuracy: 0.4762 - val_loss: 1.5480
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4310 - loss: 1.4251 - val_accuracy: 0.5238 - val_loss: 1.5327

Evaluate model

In [11]:
results = model.evaluate(X_test, y_test)

print('Test loss:', results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6190 - loss: 1.2450
Test loss: [1.2449941635131836, 0.6190476417541504]


Use model

In [17]:
i = random.randint(0,X_test.shape[0]-1)


X_prediction = X_test[i,:,:]
y_prediction = select_words[y_test[i]]

print(model.predict(np.array([X_prediction])))
print("should be", y_prediction)
print("predicted", select_words[np.argmax(model.predict(np.array([X_prediction])))])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[[0.18637513 0.66784227 0.701108   0.9382077  0.18484524]]
should be want
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
predicted want
